# Gradient Descent

### Generate Some Data

In [ ]:
import numpy as np
import dask.array as da
from dask_opt.numpy_dask import dot


def logit(y):
    return 1.0 / ( 1.0 + np.exp(-y) )


def make_y(X, beta0):
    N, M   = X.shape        
    z0     = dot(X, beta0)
    z0     = da.compute(z0)[0]  # ensure z0 is a numpy array
    scl    = S / z0.std()
    beta0 *= scl
    z0    *= scl
    y      = np.random.rand(N) < logit(z0)
    return y, z0

In [ ]:
M = 100
N = 100000
S = 2

X      = np.random.randn(N,M)
X[:,1] = 1.0
beta0  = np.random.randn(M)
y, z0 = make_y(X, beta0)
L0 = N * np.log(2.0)

### Solve with Gradient Method

In [ ]:
from dask_opt.gradient import gradient

In [ ]:
%%time
beta = gradient(X, y)

### Plot results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
z = np.dot(X,beta)
zrang = np.linspace(-6,6,200)
plt.plot(zrang,logit(zrang),'-',z,y,'o');
plt.show()

### Out-of-core simulation

In [ ]:
import os
import h5py

if not os.path.exists('X.hdf5'):
    with h5py.File('X.hdf5') as f:
        f['/X'] = X
        
f = h5py.File('X.hdf5')
X = da.from_array(f['X'], chunks=(10000, 100))

In [ ]:
from dask.diagnostics import Profiler, ResourceProfiler, visualize

with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
    beta = gradient(X, y)

In [ ]:
import bokeh.plotting as bp
bp.output_notebook()

In [ ]:
visualize([prof, rprof])

In [ ]:
if not os.path.exists('Xbig.hdf5'):
    with h5py.File('Xbig.hdf5') as f:
        out = f.require_dataset('/X', shape=(1000000, 100), dtype='f8')
        da.random.random(out.shape, chunks=(10000, 100)).store(out)    
        out[:, 0] = 1 
        
f = h5py.File('Xbig.hdf5')
X = da.from_array(f['X'], chunks=(10000, 100))
y, z0 = make_y(X, beta0)

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
    beta = gradient(X, y, max_steps=20)

In [ ]:
visualize([prof, rprof])